In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
905,tt0267913,84000000,275650703,Scooby-Doo,Freddie Prinze Jr.|Sarah Michelle Gellar|Matth...,Raja Gosnell,Get a Clue,The Mystery Inc. gang have gone their separate...,88,Mystery|Adventure|Comedy,Atlas Entertainment|Mosaic Media Group|Warner ...,6/14/2002,5.3,2002
1281,tt2017020,105000000,347434178,The Smurfs 2,Neil Patrick Harris|Christina Ricci|Katy Perry...,Raja Gosnell,Get ready to get naughty!,The evil wizard Gargamel creates a couple of m...,105,Fantasy|Family|Comedy|Animation,Columbia Pictures|Sony Pictures Animation|Kern...,7/30/2013,5.5,2013
744,tt1411238,25000000,147780440,No Strings Attached,Natalie Portman|Ashton Kutcher|Greta Gerwig|Ke...,Ivan Reitman,Friendship has its benefits.,Emma is a busy doctor who sets up a seemingly ...,108,Comedy|Romance,Paramount Pictures|Spyglass Entertainment|Kata...,1/12/2011,6.1,2011
355,tt0891592,50000000,12764201,Street Fighter: The Legend of Chun-Li,Kristin Kreuk|Chris Klein|Neal McDonough|Micha...,Andrzej Bartkowiak,Some fight for power. Some fight for us.,"When a teenager, Chun-Li witnesses the kidnapp...",97,Action|Adventure|Science Fiction|Thriller,Twentieth Century Fox Film Corporation|Hyde Pa...,2/27/2009,4.1,2009
814,tt0999913,25000000,10324441,Straw Dogs,James Marsden|Kate Bosworth|Alexander SkarsgÃ¥...,Rod Lurie,Everyone Has A Breaking Point,L.A. screenwriter David Sumner relocates with ...,110,Thriller,Battleplan Productions,9/16/2011,5.3,2011


# Предобработка

In [3]:
from typing import List, Callable
from collections import Counter

answers = {} # создадим словарь для ответов
# тут другие ваши предобработки колонок например:

# For Вопрос 21. В каком месяце за все годы суммарно вышло больше всего фильмов?
month_map = {1:  'Январь',
             2:  'Февраль',
             3:  'Март',
             4:  'Апрель',
             5:  'Май',
             6:  'Июнь',
             7:  'Июль',
             8:  'Август',
             9:  'Сентябрь',
             10: 'Октябрь',
             11: 'Ноябрь',
             12: 'Декабрь' }

# For Вопрос 26. Какие фильмы входят в один процент лучших по рейтингу?
check_answer = {
    1: ['Inside Out', 'The Dark Knight', '12 Years a Slave'],
    2: ['BloodRayne', 'The Adventures of Rocky & Bullwinkle'],
    3: ['Batman Begins', 'The Lord of the Rings: The Return of the King', 'Upside Down'],
    4: ['300', 'Lucky Number Slevin', 'Kill Bill: Vol. 1'],
    5: ['Upside Down', 'Inside Out', 'Iron Man']
}


##
# My functions here
#
def split_str_by_pipe(str_pipe: str) -> List[str]:
    """
    Split the pipe delimeted string and return
    list of words
    :param str_pipe: pipe delimeted string to split it
    :return: list of trimed string 
    """
    return split_str_by_char(str_pipe, '|')

def split_str_by_char(str_ch: str, char: str) -> List[str]:
    """
    Split string by given char into list
    and trim all words in result
    :param str_ch: string to split it
    :return: list of trimed string or empty list
    if given string is empty
    """
    if str_ch is not None and len(str_ch) > 0:
        return [ s.strip('\n\r\t ') for s in str_ch.split(char)]

    return []

# For 11. Какого жанра фильмов больше всего?
def fill_conter(str_pipe: str, func: Callable, counter:Counter):
    """
    :param func: function returns list
    :param counter: Counter to update it
    """
    counter.update(func(str_pipe))
    

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# # update data. add 'prifit' column
# data['profit'] = data[['revenue','budget']].apply(lambda row: row['revenue'] - row['budget'], axis=1 )
# # update data. add 'month' column
# data['month'] = data['release_date'].apply(lambda c: int(split_str_by_char(str(c),'/')[0]))
# # update data. add 'title_lenght' column
# data['title_lenght'] = data['original_title'].apply(lambda c: len(str(c)))
# # update data. add 'title_lenght' column
# data['overview_words_cnt'] = overview_data['overview'].apply(lambda c: len(split_str_by_char(str(c), ' ')))

# update data. add 'prifit' column
data['profit'] = data[['revenue','budget']].apply(lambda row: row['revenue'] - row['budget'], axis=1 )
# update data. add 'month' column
data['month'] = data['release_date'].apply(lambda c: int(split_str_by_char(str(c),'/')[0]))
# update data. add 'title_lenght' column
data['title_lenght'] = data['original_title'].apply(lambda c: len(str(c)))
# update data. add 'title_lenght' column
data['overview_words_cnt'] = data['overview'].apply(lambda c: len(split_str_by_char(str(c), ' ')))

# create aliaces
# profit_data = data[['imdb_id', 'profit']]
# month_data = data[['imdb_id', 'month']] 
# title_data = data[['imdb_id', 'title_lenght']] 
# overview_data = data[['imdb_id', 'overview_words_cnt']] 

# Create ganre exploded DF
# also we need the 'profit' column for 
# 12. Фильмы какого жанра чаще всего становятся прибыльными?
# solutin. Then we are taking profit_data
ganre_exploded_data = data.copy()
ganre_exploded_data['genre'] = ganre_exploded_data['genres'].apply(lambda c: split_str_by_pipe(str(c)))
ganre_exploded_data = ganre_exploded_data.explode('genre')

# Create ganre columned DF
# update df with 'ganre' columns
# if ganre is found set value 1 
# also we need the 'profit' column for 
# 12. Фильмы какого жанра чаще всего становятся прибыльными?
ganre_list = Counter()
ganre_columend_data = data.copy()
data['genres'].apply(lambda g: fill_conter(str(g), split_str_by_pipe, ganre_list))
ganres = list(ganre_list)
for g in ganres:
    ganre_columend_data[g] = ganre_columend_data['genres'].apply(lambda c: 1 if g in split_str_by_pipe(str(c)) else 0)

# Create director exploded DF
director_exploded_data = data.copy()
director_exploded_data['director'] = director_exploded_data['director'].apply(lambda c: split_str_by_pipe(str(c)))
director_exploded_data = director_exploded_data.explode('director')

# Create cast exploded DF
cast_exploded_data = data.copy()
cast_exploded_data['cast'] = cast_exploded_data['cast'].apply(lambda c: split_str_by_pipe(str(c)))
cast_exploded_data = cast_exploded_data.explode('cast')

# Create company exploded DF
# also we need the 'profit' column
company_exploded_data = data.copy()
company_exploded_data['company'] = company_exploded_data['production_companies'].apply(lambda c: split_str_by_pipe(str(c)))
company_exploded_data = company_exploded_data.explode('company')

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [4]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '...'
# если ответили верно, можете добавить комментарий со значком "+"

In [5]:
# тут пишем ваш код для решения данного вопроса:
answers['1'] = data[data['budget']==data['budget'].max()]['original_title'].values[0]
answers['1']

'Pirates of the Caribbean: On Stranger Tides'

ВАРИАНТ 2

In [6]:
# можно добавлять разные варианты решения
answers['1'] = data.loc[data['budget'].idxmax()]['original_title']
answers['1'] 

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [7]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = data.loc[data['runtime'].idxmax()]['original_title']
answers['2']

'Gods and Generals'

In [8]:
answers['2'] = data.sort_values('runtime', ascending=False).iloc[0]['original_title']
answers['2']

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [9]:
answers['3'] = data.loc[data['runtime'].idxmin()]['original_title']
answers['3']

'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [10]:
answers['4'] = round(data['runtime'].mean())
answers['4']

110

# 5. Каково медианное значение длительности фильмов? 

In [11]:
answers['5'] = round(data['runtime'].median())
answers['5']

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [12]:
# лучше код получения столбца profit вынести в Предобработку что в начале
answers['6'] = data.loc[data['profit'].idxmax()]['original_title']
answers['6']

'Avatar'

# 7. Какой фильм самый убыточный? 

In [13]:
answers['7'] = data.loc[data['profit'].idxmin()]['original_title']
answers['7']

'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [14]:
answers['8'] = data[data['profit'] > 0 ]['imdb_id'].unique().size
answers['8']

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [15]:
answers['9'] = data.loc[data[data['release_year'] == 2008 ]['profit'].idxmax()]['original_title']
answers['9']

'The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [16]:
answers['10'] = data.loc[data[data['release_year'].between(2012, 2014, inclusive=True)]['profit'].idxmin()]['original_title']
answers['10']

'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [17]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

# Create counter    
ganre_counter = Counter()

# Run apply and upadte Counter obj
# split_str_by_pipe - splits string by pipeline and returns list of word
data['genres'].apply(lambda g: fill_conter(str(g), split_str_by_pipe, ganre_counter))

answers['11'] = ganre_counter.most_common(1)[0][0]
answers['11']

'Drama'

ВАРИАНТ 2

In [18]:
# ganres - list of gannres created above
ganre_columend_data[ganres].sum().idxmax()

'Drama'

ВАРИАНТ 3

In [19]:
ganre_exploded_data['genre'].value_counts().idxmax()

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [20]:
ganre_columend_data[ganre_columend_data['profit'] > 0][ganres].sum().idxmax()

'Drama'

In [21]:
answers['12'] = ganre_exploded_data[ganre_exploded_data['profit'] > 0]['genre'].value_counts().idxmax()
answers['12']

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [22]:
answers['13'] = director_exploded_data.groupby('director')['revenue'].apply(sum).idxmax()
answers['13']

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [23]:
# merge ganre_exploded_data and director_exploded_data
# and cont max
answers['14'] = ganre_exploded_data[['imdb_id','genre']][ganre_exploded_data['genre']=='Action'] \
               .merge(director_exploded_data[['imdb_id','director']], on='imdb_id')['director'] \
               .value_counts().idxmax()
answers['14']

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [24]:
answers['15'] = cast_exploded_data[cast_exploded_data['release_year']==2012] \
                .groupby('cast')['revenue'].apply(sum).idxmax()
answers['15']

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [25]:
answers['16'] = cast_exploded_data[cast_exploded_data['budget'] > cast_exploded_data['budget'].mean()] \
                ['cast'].value_counts().idxmax()
answers['16']

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [26]:
answers['17'] = cast_exploded_data[['imdb_id','cast']][cast_exploded_data['cast']=='Nicolas Cage'] \
                .merge(ganre_exploded_data[['imdb_id','genre']], on='imdb_id') \
                ['genre'].value_counts().idxmax()
answers['17']

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [27]:
answers['18'] = company_exploded_data[(company_exploded_data['company'].str.startswith('Paramount Pictures'))]. \
                sort_values('profit', ascending=True). \
                iloc[0]['original_title']
answers['18']

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [28]:
answers['19'] = data.groupby('release_year')['revenue'].apply(sum).idxmax()
answers['19']

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [29]:
answers['20'] = company_exploded_data[(company_exploded_data['company'].str.startswith('Warner Bros'))] \
                .groupby('release_year')['profit'].apply(sum).idxmax()
answers['20']

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [30]:
answers['21'] = month_map[data.groupby('month')['imdb_id'].nunique().idxmax()]
answers['21']

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [31]:
answers['22'] = data[data['month'].isin([6,7,8])]['imdb_id'].unique().size
answers['22']

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [32]:
answers['23'] = data[['imdb_id','month']][data['month'].isin([1,2,12])] \
                .merge(director_exploded_data[['imdb_id','director']], on='imdb_id') \
                ['director'].value_counts().idxmax()
answers['23']

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [33]:
answers['24'] = company_exploded_data[['imdb_id','company']] \
               .merge(data[['imdb_id','title_lenght']], on='imdb_id') \
               .groupby('company')['title_lenght'].mean().idxmax()
answers['24']

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [34]:
answers['25'] = company_exploded_data[['imdb_id','company']] \
               .merge(data[['imdb_id','overview_words_cnt']], on='imdb_id') \
               .groupby('company')['overview_words_cnt'].mean().idxmax()
answers['25']

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [35]:
# get top titles list
top_titles_df = data[data['vote_average'] >= data['vote_average'].quantile(0.99)] \
             .sort_values('vote_average', ascending=False)[['original_title']]

# define subsets
result_id=[]
for i, lst in check_answer.items():
    # cnt found items
    found_cnt = top_titles_df[top_titles_df['original_title'].isin(lst)].shape[0]
    # if all items found store the key
    if found_cnt == len(lst):
        result_id += [i] 

#get subset by key
answers['26'] = check_answer[result_id[0] if len(result_id) == 1 else -1]
answers['26']

['Inside Out', 'The Dark Knight', '12 Years a Slave']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

In [36]:
pair_cnt = Counter()
data['cast'].apply(lambda s: pair_cnt.update(list(itertools.combinations(split_str_by_pipe(str(s)), 2))))
answers['27'] = pair_cnt.most_common(1)[0][0]
answers['27']

('Daniel Radcliffe', 'Rupert Grint')

# Submission

In [37]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 110,
 '5': 107,
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['Inside Out', 'The Dark Knight', '12 Years a Slave'],
 '27': ('Daniel Radcliffe', 'Rupert Grint')}

In [38]:
# и убедиться что ни чего не пропустил)
len(answers)

27